In [56]:
from pathlib import Path
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from sklearn import linear_model
from joblib import dump, load

In [57]:
get_data = lambda name: pd.read_csv(name).fillna(0)

In [58]:
curr_dir = Path.cwd()
parent_dir = curr_dir.parents[0]
data_dir = parent_dir / "HackZurichData"
converge_data_dir = data_dir / "CD_data_x_Converge" / "Converge"
output_data_dir = parent_dir / "machine_learning_model"
display(converge_data_dir)

PosixPath('/Users/heikernleong/Documents/localFiles/hack_zurich_2022/HackZurichData/CD_data_x_Converge/Converge')

In [59]:
sites = ["site_189","site_190","site_232","site_246","site_247"]
# name_id = "A1-Level-28"
# ts_dir = converge_data_dir / site / f"{site}_data"
# mix_design = converge_data_dir / site / f"{site}_mix_design.csv"
# mix_properties = converge_data_dir / site / f"{site}_mix_properties.csv"

In [60]:
site_dict = {
    "site" :[],
    "water_cement_ratio" : [],
    "aggregate_cement_ratio" : [],
    "carbon_estimate" : [],
    "strength" : []
}

for site in sites:
    mix_properties = converge_data_dir / site / f"{site}_mix_properties.csv"
    mix_props_df = get_data(mix_properties)
    site_dict["site"].append(site)
    site_dict["water_cement_ratio"].append(mix_props_df.iloc[4]["value"])
    site_dict["aggregate_cement_ratio"].append(mix_props_df.iloc[5]["value"])
    site_dict["carbon_estimate"].append(mix_props_df.iloc[6]["value"])
    site_dict["strength"].append(mix_props_df.iloc[2]["value"])

In [61]:
display(site_dict)
model_df = pd.DataFrame.from_dict(site_dict)
model_df["longitude"] = [35.501778,-0.127758,103.819839,-120.740135,28.0473]
model_df["latitude"] = [33.893791,51.507351,1.352083,47.751076,26.2041]
display(model_df)

{'site': ['site_189', 'site_190', 'site_232', 'site_246', 'site_247'],
 'water_cement_ratio': ['0.45', '0.35', '0.445', '0.47', '0.455'],
 'aggregate_cement_ratio': ['4.36', '4.14', '4.438', '4.71', '4.648'],
 'carbon_estimate': ['361.24', '215', '226.94', '245.33', '189.29'],
 'strength': ['45', '60', '40', '50', '50']}

,site,water_cement_ratio,aggregate_cement_ratio,carbon_estimate,strength,longitude,latitude
0,site_189,0.45,4.36,361.24,45,35.501778,33.893791
1,site_190,0.35,4.14,215,60,-0.127758,51.507351
2,site_232,0.445,4.438,226.94,40,103.819839,1.352083
3,site_246,0.47,4.71,245.33,50,-120.740135,47.751076
4,site_247,0.455,4.648,189.29,50,28.047300,26.204100


In [62]:
y = model_df["strength"]
x = model_df[["water_cement_ratio","aggregate_cement_ratio","longitude","latitude"]]
regr = linear_model.LinearRegression()
regr.fit(x, y)

print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)

regr.predict([[0.35,4.14,30,50]])

Intercept: 
 -25.84772266232892
Coefficients: 
 [-1.72674074e+02  3.09574116e+01  4.64543441e-02  3.51908978e-01]


/usr/local/anaconda3/envs/hackzurich22/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([60.86911489])

In [63]:
model_path = output_data_dir / "model.joblib"

dump(regr, model_path) 

['/Users/heikernleong/Documents/localFiles/hack_zurich_2022/machine_learning_model/model.joblib']

In [22]:
mix_props = get_data(mix_properties)
display(mix_props)

,property_description,value,units_of_measure
0,strength grade,C40/50,0
1,strength at 28 days (cylinder),40,MPa
2,strength at 28 days (cube),50,MPa
3,slump class,S4,0
4,water/cement ratio,0.455,0
5,aggregate/cement ratio,4.648,0
6,Embodied carbon estimate,189.29,kg/m3
